In [2]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import torch
from torch import nn

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics.quantile import QuantileLoss
import pytorch_forecasting.metrics

import pandas as pd

In [38]:
csv_file = open("2016_2022.csv", "a")
csv_file.write("\n")
csv_file.write("")
csv_file.close()

In [3]:
# Load Data
datas = pd.read_csv("2016_2022.csv")
data = datas[datas["Source"] == 3]
n = len(data)
time_idx = [i for i in range(n)]
data.insert(3, "Time", time_idx, True)
data = data.drop("Date", axis = 1)
print(data)

         Valeur  Source    Time
1         3.607       3       0
13        3.496       3       1
25        3.446       3       2
37        3.298       3       3
49        3.229       3       4
...         ...     ...     ...
4034625   1.931       3  338248
4034637   1.915       3  338249
4034649   1.918       3  338250
4034661   1.942       3  338251
4034673   1.998       3  338252

[338253 rows x 3 columns]


In [4]:
# define dataset
max_encoder_length = 50
max_prediction_length = 10

In [5]:
training_cutoff = data["Time"].max() - max_prediction_length
print(training_cutoff)
#print(data[lambda x: x.Time <= training_cutoff])
training = TimeSeriesDataSet(
    data[lambda x: x.Time <= training_cutoff],
    time_idx= "Time",
    target= "Valeur",
    group_ids=["Source"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_reals=["Source"],
    time_varying_known_reals=["Time"],
    time_varying_unknown_reals=["Valeur"],
)
print(training)

338242
TimeSeriesDataSet[length=338184](
	time_idx='Time',
	target='Valeur',
	group_ids=['Source'],
	weight=None,
	max_encoder_length=50,
	min_encoder_length=50,
	min_prediction_idx=0,
	min_prediction_length=10,
	max_prediction_length=10,
	static_categoricals=[],
	static_reals=['Source'],
	time_varying_known_categoricals=[],
	time_varying_known_reals=['Time'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['Valeur'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation='relu',
	method_kwargs={}
),
	categorical_encoders={'__group_id__Source': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={'Source': StandardScaler(), 'Time': StandardScaler()},
	randomize_length=None,
	predict_mode=False
)


In [6]:
# create validation and training dataset
validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=8)
print(validation)

TimeSeriesDataSet[length=60](
	time_idx='Time',
	target='Valeur',
	group_ids=['Source'],
	weight=None,
	max_encoder_length=50,
	min_encoder_length=50,
	min_prediction_idx=338184,
	min_prediction_length=10,
	max_prediction_length=10,
	static_categoricals=[],
	static_reals=['Source'],
	time_varying_known_categoricals=[],
	time_varying_known_reals=['Time'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['Valeur'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation='relu',
	method_kwargs={}
),
	categorical_encoders={'__group_id__Source': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={'Source': StandardScaler(), 'Time': StandardScaler()},
	randomize_length=None,
	predict_mode=False
)


In [7]:
# define trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", patience=1, verbose=1, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=100,
    gradient_clip_val=0.1,
    #limit_train_batches=50,
    callbacks=[lr_logger, early_stop_callback],
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alex/miniconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [8]:
# create the model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=1,
    reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Number of parameters in network: 62.0k


/home/alex/miniconda3/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/home/alex/miniconda3/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


In [46]:
    def validation_step(self, batch, batch_idx):
        X, y, idx = batch
        pred = self(X)
        val_loss = ACARMSELoss()(pred, y)

        self.log('val_loss', val_loss)
        return {"val_loss": val_loss}

In [ ]:
# find optimal learning rate (set limit_train_batches to 1.0 and log_interval = -1)
res = trainer.tuner.lr_find(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=0.3,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at /mnt/c/Users/alex4/Desktop/COURS/PIIA-Marees/PyTorch/.lr_find_4cb64ba6-2cc0-4de2-8f4b-3aaf566ee209.ckpt


In [ ]:
# fit the model
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 96    
4  | static_variable_selection          | VariableSelectionNetwork        | 1.8 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 3.8 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1.8 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 4.3 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 4.3 K 
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 4.3 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]